<a href="https://colab.research.google.com/github/Ronak1998/Model_outputs/blob/master/House_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# univariate cnn lstm example
import tensorflow.compat.v1 as tf
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import keras.utils

Using TensorFlow backend.


In [0]:
import pandas as pd
import time
from datetime import date, time, timedelta
import numpy as np
from numpy import asarray
from numpy import save
from sklearn import preprocessing
from google.colab import drive

In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [4]:
file = r'/gdrive/My Drive/Colab Notebooks/NILM/REDDhouse2_lowf_VA.csv'
df = pd.read_csv(file)
print(df)

        TimeStamp    MAIN   KTCH   LITE  ...   REFG   DISH   GARB   DIFF
0       1303104700     37      5      8  ...      6      1      0      8
1       1303104704     37      4      8  ...      6      0      0      8
2       1303104707     37      5      8  ...      6      1      0      8
3       1303104710     37      3      8  ...      7      0      0     10
4       1303104725     37      4      8  ...      7      1      0      8
...            ...    ...    ...    ...  ...    ...    ...    ...    ...
316835  1306108734    262      2      9  ...    159      0      0     87
316836  1306108737    263      2      8  ...    158      0      0     91
316837  1306108741    262      2      9  ...    158      0      0     88
316838  1306108744    263      2      9  ...    160      0      0     88
316839  1306108748    262      2      9  ...    157      0      0     90

[316840 rows x 11 columns]


In [5]:
array = df.to_numpy()
dataset = pd.DataFrame({'timestamp': array[:,0], 'Main': array[:,1], 'Kitch': array[:,2], 'Lite': array[:,3], 'Stov': array[:,4], 'Micr': array[:,5], 'Dryr': array[:,6], 'Refg': array[:,7], 'Dish': array[:,8], 'Garb': array[:,9], 'Diff': array[:,10]})
dataset = dataset.set_index(['timestamp'])
dataset.index = pd.to_datetime(dataset.index, unit='s')
dataset = dataset.resample('1min').mean()
dataset = dataset.dropna()
dataset.describe()

,Main,Kitch,Lite,Stov,Micr,Dryr,Refg,Dish,Garb,Diff
count,20034.000000,20034.000000,20034.000000,20034.000000,20034.000000,20034.000000,20034.000000,20034.000000,20034.000000,20034.000000
mean,229.984131,16.477641,26.610907,1.445228,15.354206,2.139669,79.745037,8.968314,0.081752,79.161377
std,242.080390,81.116853,45.811541,17.794712,95.882085,0.640812,85.827320,94.953205,1.542877,119.124760
min,31.000000,0.000000,2.000000,0.000000,1.562500,0.000000,1.562500,0.000000,0.000000,6.375000
25%,37.000000,1.062500,8.000000,0.062500,4.062500,2.000000,6.125000,0.000000,0.000000,14.312500
50%,255.547917,1.562500,8.625000,0.500000,4.625000,2.000000,7.000000,0.062500,0.000000,61.000000
75%,284.820772,14.125000,9.000000,0.937500,5.000000,2.250000,160.750000,0.076923,0.000000,91.500000
max,2761.937500,1079.250000,185.400000,411.000000,1926.000000,8.812500,598.187500,1255.562500,115.750000,1032.875000


In [0]:
def LSTM_pre_processing(dataset, appliance, starting, ending, value1, value2):
  for i in range(starting, ending):
    if value1 == 0:
      if dataset[appliance][i] > value2:
        dataset["Lite"][i] = 1
      else:
        dataset["Lite"][i] = 0

    else:
      if dataset[appliance][i] > value1:
        if dataset[appliance][i] < value2:
          dataset["Lite"][i] = 1
        else:
          dataset["Lite"][i] = 0
      else:
        dataset["Lite"][i] = 0


  flag = 0
  for i in range(starting+1,ending):
    if dataset["Lite"][i] == 1:
      if dataset["Lite"][i-1] == 1:
        flag+=1
      else:
        flag = 1
    else:
      flag = 0
    dataset["Kitch"][i] = flag

  continues_active_sequence = []
  temp = []
  for i in range(starting, ending):
    if dataset["Kitch"][i] > 0:
      temp1 = []
      temp1.append(dataset["Kitch"][i])
      temp1.append(dataset[appliance][i])
      temp.append(temp1)
    else:
      if len(temp) > 0:
        continues_active_sequence.append(temp)
        temp = []


  train_x = []
  train_y = []
  for i in range(1,len(continues_active_sequence)):
    temp1 = [0,0,0,0,0]
    train_x.append(temp1)
    norm = (continues_active_sequence[i][0][1])
    train_y.append(norm)
    temp1 = [0,0,0,0]
    for j in range(0,len(continues_active_sequence[i])-1):
      norm1 = (continues_active_sequence[i][j][0])
      temp1.append(norm1)
      train_x.append(temp1)
      norm2 = (continues_active_sequence[i][j+1][1])
      train_y.append(norm2)
      temp3 = []
      for k in range(1,len(temp1)):
        temp3.append(temp1[k])
      temp1 = temp3

  LSTM_X = np.asarray(train_x)
  LSTM_y = np.asarray(train_y)

  return LSTM_X, LSTM_y;

In [0]:
def CNN_pre_processing (dataset, appliance, starting, ending, value1, value2):
  for i in range(starting, ending):
    if value1 == 0:
      if dataset[appliance][i] > value2:
        dataset["Lite"][i] = 1
      else:
        dataset["Lite"][i] = 0

    else:
      if dataset[appliance][i] > value1:
        if dataset[appliance][i] < value2:
          dataset["Lite"][i] = 1
        else:
          dataset["Lite"][i] = 0
      else:
        dataset["Lite"][i] = 0

  x = dataset.values #returns a numpy array
  min_max_scaler = preprocessing.MinMaxScaler()
  x_scaled = min_max_scaler.fit_transform(x)
  dataset1 = pd.DataFrame(x_scaled)


  lite = (dataset.index[starting+1] - dataset.index[starting])
  lite = str(lite)
  main_array = []
  check = lite

  temp = []
  for i in range(starting+1,ending):
      difference = dataset.index[i] - dataset.index[i-1]
      string = str(difference)
      if string == check:
          temp1 = []
          temp2 = []
          temp1.append(dataset1[0][i])
          temp1.append(dataset["Lite"][i])
          temp.append(temp1)
      else:
          main_array.append(temp)
          temp = []



  final_array = []
  for i in range(0,len(main_array)):
      length = len(main_array[i])
      modu = length%20
      if modu < 10:
          temp = []
          for j in range(0,len(main_array[i])-modu):
              temp.append(main_array[i][j])
          final_array.append(temp)
      else:
          temp = main_array[i]
          for j in range(0,20 - modu):
              array = [0,0]
              temp.append(array)
          final_array.append(temp)

  test_array = []
  for i in range(0,len(final_array)):
      temp = []
      for j in range(0,10):
          temp.append([0,0])
      for j in range(0,len(final_array[i])):
          temp.append(final_array[i][j])
      for j in range(0,10):
          temp.append([0,0])
      test_array.append(temp)



  training_set = []
  for i in range(0,len(test_array)):
      for j in range(0,len(test_array[i])-20):
          x_y = []
          temp = []
          for k in range(j,j+20):
              temp.append(test_array[i][k][0])
          x_y.append(temp)
          x_y.append(test_array[i][j+10][1])
          training_set.append(x_y)


  X = []
  y = []
  for i in range(0,len(training_set)):
    lets = training_set[i][0]
    lets = np.asarray(lets)
    X.append(lets)
    y.append(training_set[i][1])


  X = np.asarray(X)
  y = np.asarray(y)
  from keras.utils import to_categorical
  y = to_categorical(y)



  return X,y;

In [0]:
LSTM_X_refg,LSTM_y_refg = LSTM_pre_processing(dataset, "Refg", 0,  (len(dataset.index) - 6000), 0, 100)

LSTM_X_micr1,LSTM_y_micr1 = LSTM_pre_processing(dataset, "Micr", 6000,  (len(dataset.index)), 20, 200)

LSTM_X_micr2,LSTM_y_micr2 = LSTM_pre_processing(dataset, "Micr", 6000,  (len(dataset.index)), 0, 900)

LSTM_X_dish1,LSTM_y_dish1 = LSTM_pre_processing(dataset, "Dish", 6000,  (len(dataset.index)), 100, 300)

LSTM_X_dish2,LSTM_y_dish2 = LSTM_pre_processing(dataset, "Dish", 6000,  (len(dataset.index)), 0, 900)

In [0]:
X_refg, y_refg = CNN_pre_processing(dataset, "Refg", 0, (len(dataset.index)-6000), 0, 100)

X_micr1, y_micr1 = CNN_pre_processing(dataset, "Micr", 6000, (len(dataset.index)), 20, 100)

X_micr2, y_micr2 = CNN_pre_processing(dataset, "Micr", 6000, (len(dataset.index)), 0, 900)

X_dish1, y_dish1 = CNN_pre_processing(dataset, "Dish", 6000, (len(dataset.index)), 100, 300)

X_dish2, y_dish2 = CNN_pre_processing(dataset, "Dish", 6000, (len(dataset.index)), 0, 900)

In [10]:
print(len(X_dish1), len(LSTM_X_dish1))

13780 71


In [0]:
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional

In [0]:
n_features = 1
#n_seq = 2
n_steps = 20
X_refg = X_refg.reshape((X_refg.shape[0], n_steps, n_features))

X_micr1 = X_micr1.reshape((X_micr1.shape[0], n_steps, n_features))

X_micr2 = X_micr2.reshape((X_micr2.shape[0], n_steps, n_features))

X_dish1 = X_dish1.reshape((X_dish1.shape[0], n_steps, n_features))

X_dish2 = X_dish2.reshape((X_dish2.shape[0], n_steps, n_features))


In [13]:
model_dish1 = Sequential()
model_dish1.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps,n_features)))
model_dish1.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model_dish1.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Dropout(0.5))
model_dish1.add(MaxPooling1D(pool_size=2))
model_dish1.add(Flatten())
model_dish1.add(Dense(100, activation='relu'))
model_dish1.add(Dense(2, activation='softmax'))
model_dish1.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

model_dish1.fit(X_dish1, y_dish1, epochs=20, validation_split=0.1, verbose=1)

Train on 12402 samples, validate on 1378 samples
Epoch 1/20
12402/12402 [==============================] - 3s 230us/step - loss: 0.0516 - accuracy: 0.9941 - val_loss: 0.0019 - val_accuracy: 1.0000
Epoch 2/20
12402/12402 [==============================] - 3s 205us/step - loss: 0.0296 - accuracy: 0.9943 - val_loss: 0.0057 - val_accuracy: 1.0000
Epoch 3/20
12402/12402 [==============================] - 3s 202us/step - loss: 0.0228 - accuracy: 0.9943 - val_loss: 0.0053 - val_accuracy: 1.0000
Epoch 4/20
12402/12402 [==============================] - 3s 203us/step - loss: 0.0212 - accuracy: 0.9941 - val_loss: 0.0055 - val_accuracy: 1.0000
Epoch 5/20
12402/12402 [==============================] - 2s 198us/step - loss: 0.0206 - accuracy: 0.9942 - val_loss: 0.0045 - val_accuracy: 1.0000
Epoch 6/20
12402/12402 [==============================] - 3s 205us/step - loss: 0.0186 - accuracy: 0.9945 - val_loss: 0.0041 - val_accuracy: 1.0000
Epoch 7/20
12402/12402 [==============================] - 3s 20

In [14]:
model_dish2 = Sequential()
model_dish2.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps,n_features)))
model_dish2.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model_dish2.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Dropout(0.5))
model_dish2.add(MaxPooling1D(pool_size=2))
model_dish2.add(Flatten())
model_dish2.add(Dense(100, activation='relu'))
model_dish2.add(Dense(2, activation='softmax'))
model_dish2.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

model_dish2.fit(X_dish2, y_dish2, epochs=20, validation_split=0.1, verbose=1)

Train on 12402 samples, validate on 1378 samples
Epoch 1/20
12402/12402 [==============================] - 3s 228us/step - loss: 0.0375 - accuracy: 0.9947 - val_loss: 7.6996e-04 - val_accuracy: 1.0000
Epoch 2/20
12402/12402 [==============================] - 3s 210us/step - loss: 0.0094 - accuracy: 0.9955 - val_loss: 0.0057 - val_accuracy: 0.9971
Epoch 3/20
12402/12402 [==============================] - 3s 218us/step - loss: 0.0048 - accuracy: 0.9981 - val_loss: 1.8258e-04 - val_accuracy: 1.0000
Epoch 4/20
12402/12402 [==============================] - 3s 222us/step - loss: 0.0044 - accuracy: 0.9983 - val_loss: 6.1430e-04 - val_accuracy: 1.0000
Epoch 5/20
12402/12402 [==============================] - 3s 225us/step - loss: 0.0041 - accuracy: 0.9986 - val_loss: 5.6947e-05 - val_accuracy: 1.0000
Epoch 6/20
12402/12402 [==============================] - 3s 219us/step - loss: 0.0037 - accuracy: 0.9987 - val_loss: 0.0025 - val_accuracy: 0.9985
Epoch 7/20
12402/12402 [=======================

In [15]:
model_micr1 = Sequential()
model_micr1.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps,n_features)))
model_micr1.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model_micr1.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Dropout(0.5))
model_micr1.add(MaxPooling1D(pool_size=2))
model_micr1.add(Flatten())
model_micr1.add(Dense(100, activation='relu'))
model_micr1.add(Dense(2, activation='softmax'))
model_micr1.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

model_micr1.fit(X_micr1, y_micr1, epochs=20, validation_split=0.1, verbose=1)

Train on 12402 samples, validate on 1378 samples
Epoch 1/20
12402/12402 [==============================] - 3s 221us/step - loss: 0.2976 - accuracy: 0.8962 - val_loss: 0.2177 - val_accuracy: 0.9187
Epoch 2/20
12402/12402 [==============================] - 3s 207us/step - loss: 0.2343 - accuracy: 0.9066 - val_loss: 0.1626 - val_accuracy: 0.9202
Epoch 3/20
12402/12402 [==============================] - 3s 209us/step - loss: 0.1925 - accuracy: 0.9222 - val_loss: 0.1378 - val_accuracy: 0.9448
Epoch 4/20
12402/12402 [==============================] - 3s 218us/step - loss: 0.1680 - accuracy: 0.9283 - val_loss: 0.1737 - val_accuracy: 0.9202
Epoch 5/20
12402/12402 [==============================] - 3s 221us/step - loss: 0.1624 - accuracy: 0.9295 - val_loss: 0.1953 - val_accuracy: 0.9136
Epoch 6/20
12402/12402 [==============================] - 3s 217us/step - loss: 0.1555 - accuracy: 0.9325 - val_loss: 0.1619 - val_accuracy: 0.9492
Epoch 7/20
12402/12402 [==============================] - 3s 21

In [16]:
model_micr2 = Sequential()
model_micr2.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps,n_features)))
model_micr2.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model_micr2.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Dropout(0.5))
model_micr2.add(MaxPooling1D(pool_size=2))
model_micr2.add(Flatten())
model_micr2.add(Dense(100, activation='relu'))
model_micr2.add(Dense(2, activation='softmax'))
model_micr2.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

model_micr2.fit(X_micr2, y_micr2, epochs=20, validation_split=0.1, verbose=1)

Train on 12402 samples, validate on 1378 samples
Epoch 1/20
12402/12402 [==============================] - 3s 216us/step - loss: 0.0449 - accuracy: 0.9955 - val_loss: 0.0269 - val_accuracy: 0.9906
Epoch 2/20
12402/12402 [==============================] - 2s 201us/step - loss: 0.0097 - accuracy: 0.9961 - val_loss: 0.0092 - val_accuracy: 0.9942
Epoch 3/20
12402/12402 [==============================] - 3s 207us/step - loss: 0.0049 - accuracy: 0.9981 - val_loss: 0.0765 - val_accuracy: 0.9906
Epoch 4/20
12402/12402 [==============================] - 3s 207us/step - loss: 0.0044 - accuracy: 0.9981 - val_loss: 0.0052 - val_accuracy: 0.9978
Epoch 5/20
12402/12402 [==============================] - 3s 207us/step - loss: 0.0040 - accuracy: 0.9984 - val_loss: 0.0045 - val_accuracy: 0.9985
Epoch 6/20
12402/12402 [==============================] - 2s 198us/step - loss: 0.0021 - accuracy: 0.9992 - val_loss: 0.0041 - val_accuracy: 0.9985
Epoch 7/20
12402/12402 [==============================] - 2s 19

In [17]:
model_refg = Sequential()
model_refg.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps,n_features)))
model_refg.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model_refg.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Dropout(0.5))
model_refg.add(MaxPooling1D(pool_size=2))
model_refg.add(Flatten())
model_refg.add(Dense(100, activation='relu'))
model_refg.add(Dense(2, activation='softmax'))
model_refg.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])


model_refg.fit(X_refg, y_refg, epochs=20, validation_split=0.1, verbose=1)



Train on 11556 samples, validate on 1284 samples
Epoch 1/20
11556/11556 [==============================] - 3s 231us/step - loss: 0.3145 - accuracy: 0.8560 - val_loss: 0.2532 - val_accuracy: 0.8949
Epoch 2/20
11556/11556 [==============================] - 2s 212us/step - loss: 0.2108 - accuracy: 0.9050 - val_loss: 0.2292 - val_accuracy: 0.9120
Epoch 3/20
11556/11556 [==============================] - 2s 209us/step - loss: 0.1793 - accuracy: 0.9193 - val_loss: 0.2622 - val_accuracy: 0.9097
Epoch 4/20
11556/11556 [==============================] - 2s 211us/step - loss: 0.1611 - accuracy: 0.9329 - val_loss: 0.1748 - val_accuracy: 0.9322
Epoch 5/20
11556/11556 [==============================] - 3s 218us/step - loss: 0.1577 - accuracy: 0.9351 - val_loss: 0.1549 - val_accuracy: 0.9424
Epoch 6/20
11556/11556 [==============================] - 2s 214us/step - loss: 0.1406 - accuracy: 0.9414 - val_loss: 0.2039 - val_accuracy: 0.8855
Epoch 7/20
11556/11556 [==============================] - 3s 21

In [0]:
n_steps_lstm = 5
n_features_lstm = 1
LSTM_X_refg = LSTM_X_refg.reshape((LSTM_X_refg.shape[0], LSTM_X_refg.shape[1], n_features_lstm))

LSTM_X_micr1 = LSTM_X_micr1.reshape((LSTM_X_micr1.shape[0], LSTM_X_micr1.shape[1], n_features_lstm))

LSTM_X_micr2 = LSTM_X_micr2.reshape((LSTM_X_micr2.shape[0], LSTM_X_micr2.shape[1], n_features_lstm))

LSTM_X_dish1 = LSTM_X_dish1.reshape((LSTM_X_dish1.shape[0], LSTM_X_dish1.shape[1], n_features_lstm))

LSTM_X_dish2 = LSTM_X_dish2.reshape((LSTM_X_dish2.shape[0], LSTM_X_dish2.shape[1], n_features_lstm))

In [19]:
model_lstm_dish1 = Sequential()
model_lstm_dish1.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_lstm, n_features_lstm)))
model_lstm_dish1.add(LSTM(50, activation='relu'))
model_lstm_dish1.add(Dense(1))
model_lstm_dish1.compile(optimizer='adam', loss='mse')

model_lstm_dish1.fit(LSTM_X_dish1, LSTM_y_dish1, epochs=30, verbose=1)

Epoch 1/30
71/71 [==============================] - 1s 9ms/step - loss: 53954.4034
Epoch 2/30
71/71 [==============================] - 0s 397us/step - loss: 53933.5890
Epoch 3/30
71/71 [==============================] - 0s 339us/step - loss: 53910.3919
Epoch 4/30
71/71 [==============================] - 0s 328us/step - loss: 53886.8888
Epoch 5/30
71/71 [==============================] - 0s 324us/step - loss: 53862.6629
Epoch 6/30
71/71 [==============================] - 0s 318us/step - loss: 53833.2732
Epoch 7/30
71/71 [==============================] - 0s 304us/step - loss: 53797.9784
Epoch 8/30
71/71 [==============================] - 0s 311us/step - loss: 53750.5062
Epoch 9/30
71/71 [==============================] - 0s 342us/step - loss: 53680.6591
Epoch 10/30
71/71 [==============================] - 0s 392us/step - loss: 53579.0358
Epoch 11/30
71/71 [==============================] - 0s 399us/step - loss: 53443.1440
Epoch 12/30
71/71 [==============================] - 0s 366us/ste

In [20]:
model_lstm_dish2 = Sequential()
model_lstm_dish2.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_lstm, n_features_lstm)))
model_lstm_dish2.add(LSTM(50, activation='relu'))
model_lstm_dish2.add(Dense(1))
model_lstm_dish2.compile(optimizer='adam', loss='mse')

model_lstm_dish2.fit(LSTM_X_dish2, LSTM_y_dish2, epochs=30, verbose=1)

Epoch 1/30
73/73 [==============================] - 1s 8ms/step - loss: 1411897.5411
Epoch 2/30
73/73 [==============================] - 0s 322us/step - loss: 1411678.2260
Epoch 3/30
73/73 [==============================] - 0s 357us/step - loss: 1411472.6233
Epoch 4/30
73/73 [==============================] - 0s 354us/step - loss: 1411255.4914
Epoch 5/30
73/73 [==============================] - 0s 349us/step - loss: 1411019.5428
Epoch 6/30
73/73 [==============================] - 0s 335us/step - loss: 1410718.7808
Epoch 7/30
73/73 [==============================] - 0s 324us/step - loss: 1410341.1113
Epoch 8/30
73/73 [==============================] - 0s 327us/step - loss: 1409808.7038
Epoch 9/30
73/73 [==============================] - 0s 369us/step - loss: 1408955.1301
Epoch 10/30
73/73 [==============================] - 0s 320us/step - loss: 1407376.7055
Epoch 11/30
73/73 [==============================] - 0s 332us/step - loss: 1405137.9418
Epoch 12/30
73/73 [========================

In [21]:
model_lstm_micr1 = Sequential()
model_lstm_micr1.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_lstm, n_features_lstm)))
model_lstm_micr1.add(LSTM(50, activation='relu'))
model_lstm_micr1.add(Dense(1))
model_lstm_micr1.compile(optimizer='adam', loss='mse')

model_lstm_micr1.fit(LSTM_X_micr1, LSTM_y_micr1, epochs=30, verbose=1)

Epoch 1/30
1390/1390 [==============================] - 1s 653us/step - loss: 1095.6416
Epoch 2/30
1390/1390 [==============================] - 0s 231us/step - loss: 405.7699
Epoch 3/30
1390/1390 [==============================] - 0s 229us/step - loss: 195.3734
Epoch 4/30
1390/1390 [==============================] - 0s 252us/step - loss: 103.3076
Epoch 5/30
1390/1390 [==============================] - 0s 250us/step - loss: 97.9409
Epoch 6/30
1390/1390 [==============================] - 0s 231us/step - loss: 96.6452
Epoch 7/30
1390/1390 [==============================] - 0s 238us/step - loss: 100.4048
Epoch 8/30
1390/1390 [==============================] - 0s 229us/step - loss: 105.7617
Epoch 9/30
1390/1390 [==============================] - 0s 218us/step - loss: 97.6020
Epoch 10/30
1390/1390 [==============================] - 0s 233us/step - loss: 97.9099
Epoch 11/30
1390/1390 [==============================] - 0s 225us/step - loss: 95.6269
Epoch 12/30
1390/1390 [======================

In [22]:
model_lstm_micr2 = Sequential()
model_lstm_micr2.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_lstm, n_features_lstm)))
model_lstm_micr2.add(LSTM(50, activation='relu'))
model_lstm_micr2.add(Dense(1))
model_lstm_micr2.compile(optimizer='adam', loss='mse')

model_lstm_micr2.fit(LSTM_X_micr2, LSTM_y_micr2, epochs=30, verbose=1)

Epoch 1/30
66/66 [==============================] - 1s 9ms/step - loss: 2233625.7500
Epoch 2/30
66/66 [==============================] - 0s 402us/step - loss: 2233562.0455
Epoch 3/30
66/66 [==============================] - 0s 367us/step - loss: 2233482.0758
Epoch 4/30
66/66 [==============================] - 0s 345us/step - loss: 2233393.2727
Epoch 5/30
66/66 [==============================] - 0s 339us/step - loss: 2233289.3939
Epoch 6/30
66/66 [==============================] - 0s 368us/step - loss: 2233163.2273
Epoch 7/30
66/66 [==============================] - 0s 347us/step - loss: 2233002.6970
Epoch 8/30
66/66 [==============================] - 0s 354us/step - loss: 2232805.7424
Epoch 9/30
66/66 [==============================] - 0s 339us/step - loss: 2232563.6364
Epoch 10/30
66/66 [==============================] - 0s 390us/step - loss: 2232291.7273
Epoch 11/30
66/66 [==============================] - 0s 417us/step - loss: 2231894.5000
Epoch 12/30
66/66 [========================

In [23]:
model_lstm_refg = Sequential()
model_lstm_refg.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_lstm, n_features_lstm)))
model_lstm_refg.add(LSTM(50, activation='relu'))
model_lstm_refg.add(Dense(1))
model_lstm_refg.compile(optimizer='adam', loss='mse')



model_lstm_refg.fit(LSTM_X_refg, LSTM_y_refg, epochs=30, verbose=1)



Epoch 1/30
5887/5887 [==============================] - 2s 319us/step - loss: 10100.6926
Epoch 2/30
5887/5887 [==============================] - 1s 222us/step - loss: 1904.0598
Epoch 3/30
5887/5887 [==============================] - 1s 230us/step - loss: 1916.1965
Epoch 4/30
5887/5887 [==============================] - 1s 223us/step - loss: 1892.4425
Epoch 5/30
5887/5887 [==============================] - 1s 227us/step - loss: 1905.8697
Epoch 6/30
5887/5887 [==============================] - 1s 223us/step - loss: 1932.8228
Epoch 7/30
5887/5887 [==============================] - 1s 226us/step - loss: 1900.8362
Epoch 8/30
5887/5887 [==============================] - 1s 228us/step - loss: 1925.7578
Epoch 9/30
5887/5887 [==============================] - 1s 234us/step - loss: 1913.3325
Epoch 10/30
5887/5887 [==============================] - 1s 232us/step - loss: 1909.7682
Epoch 11/30
5887/5887 [==============================] - 1s 236us/step - loss: 1894.9351
Epoch 12/30
5887/5887 [======

In [0]:
def CNN_pre_processing_test (dataset, appliance, starting, ending, value1, value2):
  for i in range(starting, ending):
    if value1 == 0:
      if dataset[appliance][i] > value2:
        dataset["Lite"][i] = 1
      else:
        dataset["Lite"][i] = 0

    else:
      if dataset[appliance][i] > value1:
        if dataset[appliance][i] < value2:
          dataset["Lite"][i] = 1
        else:
          dataset["Lite"][i] = 0
      else:
        dataset["Lite"][i] = 0

  x = dataset.values #returns a numpy array
  min_max_scaler = preprocessing.MinMaxScaler()
  x_scaled = min_max_scaler.fit_transform(x)
  dataset1 = pd.DataFrame(x_scaled)


  lite = (dataset.index[starting+1] - dataset.index[starting])
  lite = str(lite)
  main_array = []
  check = lite

  temp = []
  for i in range(starting+1,ending):
      difference = dataset.index[i] - dataset.index[i-1]
      string = str(difference)
      if string == check:
          temp1 = []
          temp2 = []
          temp1.append(dataset1[0][i])
          temp1.append(dataset["Lite"][i])
          temp.append(temp1)
      else:
          main_array.append(temp)
          temp = []



  final_array = []
  for i in range(0,len(main_array)):
      length = len(main_array[i])
      modu = length%20
      if modu < 10:
          temp = []
          for j in range(0,len(main_array[i])-modu):
              temp.append(main_array[i][j])
          final_array.append(temp)
      else:
          temp = main_array[i]
          for j in range(0,20 - modu):
              array = [0,0]
              temp.append(array)
          final_array.append(temp)



  test_array = []
  for i in range(0,len(final_array)):
      temp = []
      for j in range(0,10):
          temp.append([0,0])
      for j in range(0,len(final_array[i])):
          temp.append(final_array[i][j])
      for j in range(0,10):
          temp.append([0,0])
      test_array.append(temp)



  training_set = []
  for i in range(0,len(test_array)):
      for j in range(0,len(test_array[i])-20):
          x_y = []
          temp = []
          for k in range(j,j+20):
              temp.append(test_array[i][k][0])
          x_y.append(temp)
          x_y.append(test_array[i][j+10][1])
          training_set.append(x_y)


  X = []
  y = []
  for i in range(0,len(training_set)):
    lets = training_set[i][0]
    lets = np.asarray(lets)
    X.append(lets)
    y.append(training_set[i][1])


  X = np.asarray(X)
  y = np.asarray(y)
  from keras.utils import to_categorical
  y = to_categorical(y)



  return X,y;

In [0]:
def CNN_pre_processing_finaly (dataset, appliance, starting, ending, value1, value2):
  for i in range(starting, ending):
    if value1 == 0:
      if dataset[appliance][i] > value2:
        dataset["Lite"][i] = 1
      else:
        dataset["Lite"][i] = 0

    else:
      if dataset[appliance][i] > value1:
        if dataset[appliance][i] < value2:
          dataset["Lite"][i] = 1
        else:
          dataset["Lite"][i] = 0
      else:
        dataset["Lite"][i] = 0

  lite = (dataset.index[starting+1] - dataset.index[starting])
  lite = str(lite)
  main_array = []
  check = lite

  temp = []
  for i in range(starting+1,ending):
      difference = dataset.index[i] - dataset.index[i-1]
      string = str(difference)
      if string == check:
          temp1 = []
          temp2 = []
          temp1.append(dataset[appliance][i])
          temp1.append(dataset["Lite"][i])
          temp.append(temp1)
      else:
          main_array.append(temp)
          temp = []



  final_array = []
  for i in range(0,len(main_array)):
      length = len(main_array[i])
      modu = length%20
      if modu < 10:
          temp = []
          for j in range(0,len(main_array[i])-modu):
              temp.append(main_array[i][j])
          final_array.append(temp)
      else:
          temp = main_array[i]
          for j in range(0,20 - modu):
              array = [0,0]
              temp.append(array)
          final_array.append(temp)


  final_y = []
  for i in range(0,len(final_array)):
    for j in range(0,len(final_array[i])):
      final_y.append(final_array[i][j][0])
  len(final_y)

  return final_y;

In [0]:
X_test_refg, y_test_refg = CNN_pre_processing_test(dataset, "Refg", (len(dataset.index) - 6000), len(dataset.index), 0, 100)

X_test_micr1, y_test_micr1 = CNN_pre_processing_test(dataset, "Micr", 0, 6000, 20, 100)

X_test_micr2, y_test_micr2 = CNN_pre_processing_test(dataset, "Micr", 0, 6000, 0, 900)

X_test_dish1, y_test_dish1 = CNN_pre_processing_test(dataset, "Dish", 0, 6000, 100, 300)

X_test_dish2, y_test_dish2 = CNN_pre_processing_test(dataset, "Dish", 0, 6000, 0, 900)


In [0]:
final_y = CNN_pre_processing_finaly(dataset, "Refg",  (len(dataset.index) - 6000), len(dataset.index), 0, 100)

final_y_micr1 = CNN_pre_processing_finaly(dataset, "Micr", 0, 6000, 20, 100)

final_y_micr2 = CNN_pre_processing_finaly(dataset, "Micr", 0, 6000, 0, 900)

final_y_dish1 = CNN_pre_processing_finaly(dataset, "Dish", 0, 6000, 100, 300)

final_y_dish2 = CNN_pre_processing_finaly(dataset, "Dish", 0, 6000, 0, 900)

In [0]:
#X_test_micr = X_test_micr.reshape((X_test_micr.shape[0], n_steps, n_features))
#_, accuracy = model_micr.evaluate(X_test_micr, y_test_micr,  verbose=0)

#print(accuracy)

In [0]:
X_test_refg = X_test_refg.reshape((X_test_refg.shape[0], 20, 1))
_, accuracy = model_refg.evaluate(X_test_refg, y_test_refg,  verbose=0)


In [30]:
print(accuracy)

0.9505208134651184


In [31]:
print(len(X_test_refg), len(y_test_refg), len(final_y))
print(len(X_test_micr1), len(y_test_micr1), len(final_y_micr1))
print(len(X_test_micr2), len(y_test_micr2), len(final_y_micr2))
print(len(X_test_dish1), len(y_test_dish1), len(final_y_dish1))
print(len(X_test_dish2), len(y_test_dish2), len(final_y_dish2))

5760 5760 5760
5580 5580 5580
5580 5580 5580
5580 5580 5580
5580 5580 5580


In [0]:
def testset_lstm (lite, deadload):
  qwerty = []
  flag = 0
  temp = [0,0,0,0,0]
  for i in range(0,len(lite)):
    if lite[i] == 0:
      qwerty.append([deadload])
      flag = 0
      temp = [0,0,0,0,0]
    else:
      flag+=1
      temp2 = []
      qwerty.append(temp)
      temp.append(flag)
      for j in range(len(temp)-5,len(temp)):
        temp2.append(temp[j])
      temp = temp2
  
  for i in range(0,len(qwerty)):
    if len(qwerty) > 2:
      good = []
      for j in range(0,len(qwerty[i])-1):
        good.append(qwerty[i][j])
      qwerty[i] = good
    else:
      qwerty[i] = qwerty[i].append(deadload)


  final = []
  for i in range(0,len(qwerty)):
    if len(qwerty[i]) > 2:
      good = []
      for j in range(0,len(qwerty[i])):
        good.append(qwerty[i][j])
      final.append(good)
    else:
      temp = []
      temp.append(deadload)
      final.append(temp)

  return final;


In [0]:
lite_refg = []
for i in range(0,len(X_test_refg)):
  x_input = X_test_refg[i].reshape(1,20,1)
  yhat = model_refg.predict_classes(x_input,verbose=0)
  lite_refg.append(yhat[0])

In [0]:
lite_micr1 = []
for i in range(0,len(X_test_micr1)):
  x_input = X_test_micr1[i].reshape(1,20,1)
  yhat = model_micr1.predict_classes(x_input,verbose=0)
  lite_micr1.append(yhat[0])

In [0]:
lite_micr2 = []
for i in range(0,len(X_test_micr2)):
  x_input = X_test_micr2[i].reshape(1,20,1)
  yhat = model_micr2.predict_classes(x_input,verbose=0)
  lite_micr2.append(yhat[0])

In [0]:
lite_dish1 = []
for i in range(0,len(X_test_dish1)):
  x_input = X_test_dish1[i].reshape(1,20,1)
  yhat = model_dish1.predict_classes(x_input,verbose=0)
  lite_dish1.append(yhat[0])

In [0]:
lite_dish2 = []
for i in range(0,len(X_test_dish2)):
  x_input = X_test_dish2[i].reshape(1,20,1)
  yhat = model_dish2.predict_classes(x_input,verbose=0)
  lite_dish2.append(yhat[0])

In [0]:
Refg = testset_lstm(lite_refg, 6)
Micr1 = testset_lstm(lite_micr1, 4)
Micr2 = testset_lstm(lite_micr2, 4)
Dish1 = testset_lstm(lite_dish1, 0)
Dish2 = testset_lstm(lite_dish2, 0)

In [39]:
lol = 0
error = 0
total = 0
tot = 0
tot_m = 0
total_m = 0
tot_d = 0
total_d = 0

for i in range(0,len(Refg)):
  if len(Refg[i]) < 2:
    predic = Refg[i][0]
  else:
    x_input = np.asarray(Refg[i])
    x_input = x_input.reshape((1,5,1))
    yhat = model_lstm_refg.predict(x_input, verbose=0)
    predic = yhat[0][0]
  print(predic, final_y[i])
  tot += predic
  total += final_y[i]
  #lol = (abs(predic+0.1 - final_y[i]+0.1)/ (final_y[i]+0.1))/7980




for i in range(0,len(Micr2)):

  if len(Micr2[i]) < 2:
    if len(Micr1[i]) < 2:
      predic = Micr2[i][0]
    else:
      x_input = np.asarray(Micr1[i])
      x_input = x_input.reshape((1,5,1))
      yhat = model_lstm_micr1.predict(x_input, verbose=0)
      predic = yhat[0][0]
      #print("Micr1")
  else:
    x_input = np.asarray(Micr2[i])
    x_input = x_input.reshape((1,5,1))
    yhat = model_lstm_micr2.predict(x_input, verbose=0)
    predic = 1400
    #print("Micr2")
  #print(predic, final_y_micr2[i])
  tot_m += predic
  total_m += final_y_micr2[i]



for i in range(0,len(Dish2)):

  if len(Dish2[i]) < 2:
    if len(Dish1[i]) < 2:
      predic = Dish2[i][0]
    else:
      x_input = np.asarray(Dish1[i])
      x_input = x_input.reshape((1,5,1))
      yhat = model_lstm_dish1.predict(x_input, verbose=0)
      predic = yhat[0][0]
      predic = 250
      #print("dish1")
  else:
    x_input = np.asarray(Dish2[i])
    x_input = x_input.reshape((1,5,1))
    yhat = model_lstm_dish2.predict(x_input, verbose=0)
    predic = yhat[0][0]
    predic = 1150
    #print("dish2")

  #print(predic, final_y_dish2[i])
  tot_d += predic
  total_d += final_y_dish2[i]

  



Streaming output truncated to the last 5000 lines.
6 6.0
6 6.0
6 6.0
6 6.0625
6 6.0
6 6.0
6 16.5
180.28606 183.375
179.34518 171.6875
177.89746 168.4375
178.47676 167.9375
180.46274 167.0
179.43991 166.26666666666668
180.0372 165.5
179.60225 164.0625
180.46336 163.875
181.35425 163.0
181.4454 162.6875
181.15443 161.75
180.53993 161.3125
179.87018 160.25
179.29552 159.375
178.84332 159.0625
178.58954 158.6
178.52882 157.9375
178.67633 157.75
179.01088 157.25
179.54988 157.5625
180.16205 157.0625
180.82428 157.0625
181.51875 156.1875
6 72.0
6 6.0
6 6.0
6 6.0
6 6.0
6 6.0
6 6.0625
6 6.0
6 6.0
6 6.0
6 6.0
6 6.0
6 6.0
6 6.0
6 6.0
6 6.0
6 6.0625
6 6.0
6 6.0
6 6.0
6 6.0
6 6.0625
6 6.0
6 6.0
6 6.0
6 6.0
6 6.0
6 6.0
6 6.0
6 39.25
6 180.125
6 170.625
6 167.5625
6 166.8125
6 166.0
6 164.6875
6 164.0
180.28606 163.5625
179.34518 162.6875
177.89746 162.25
6 161.8
6 161.5625
180.28606 160.875
179.34518 159.75
177.89746 159.9375
178.47676 159.375
180.46274 158.5625
179.43991 158.1875
180.0372 158.5625

In [40]:
print(tot_d, total_d)

70800 71093.97709275996


In [41]:
print(tot, total)
print(tot_m, total_m)
print(tot_d, total_d)


497866.74014282227 503133.6493717063
55479.31491088867 66285.93995098035
70800 71093.97709275996


In [0]:
erro = abs((total + total_m + total_d) - (tot + tot_m + tot_d))/(total + total_m + total_d)

In [43]:
1 - (abs(tot - total)/total)

0.9895317889482026

In [44]:
(1-erro)*100

97.4446269025441

In [45]:
((1 - (abs(tot_d - total_d)/ total_d)) + (1 - (abs(tot_m - total_m)/ total_m)) + (1 - (abs(tot - total)/ total)))/3

0.9407887711902942

In [46]:
for i in range(0, len(lite_refg)):
  if lite_refg[i] == 1:
    print(final_y[i])

170.4
170.3125
169.8125
169.125
168.5
168.125
167.4375
167.625
167.8125
167.25
166.8
166.0
165.875
165.4375
165.0
164.5
162.9375
162.5625
162.875
162.2
162.3125
162.0625
161.4375
160.875
161.5
161.0625
160.8125
160.6875
160.0625
160.2
159.875
159.6875
159.25
159.8125
159.0
159.8125
159.1875
158.8125
158.1875
158.4375
158.125
157.8125
157.875
157.6875
157.4375
157.0625
156.875
156.875
157.1875
156.8125
156.6875
157.0
157.25
86.125
181.375
172.125
169.4375
169.125
167.625
167.375
166.5
165.75
165.4375
164.6875
164.5
163.875
163.0625
163.125
162.4375
161.875
161.9375
161.1875
160.6
160.625
160.625
160.125
159.3125
159.875
159.5625
159.1875
158.8125
158.625
180.86666666666667
173.875
171.25
170.25
169.5625
168.375
167.625
166.4
165.3125
164.75
163.9375
163.1875
162.8125
162.3125
162.1875
162.125
161.5625
160.93333333333334
161.0
160.64705882352942
160.33333333333334
160.0
159.75
159.5625
159.625
182.1875
173.3125
170.73333333333332
170.3125
169.3125
168.625
167.4375
166.8125
166.1875
165.8

In [47]:
for i in range(0,len(final_y_micr2)):
  if final_y_micr2[i] > 100:
    print(final_y_micr2[i], lite_micr2[i], lite_micr1[i])

642.5625 0 0
125.9375 0 0
1867.4 1 0
137.875 0 0
1137.9333333333334 1 0
708.25 0 0
1070.8125 0 0
124.75 0 0
1888.5 1 0
142.9375 0 0
1556.625 1 0
1765.7333333333333 1 0
962.9375 1 0
733.25 0 0
1899.8666666666666 1 0
526.125 0 0
980.625 0 0
788.0 0 0
